In [2]:
#importing the libraries which will be used later
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date, time
from datetime import datetime as dt

In [3]:
#Getting the turnstile data from MTA

df1 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220507.txt")

df2 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220514.txt")

df3 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220521.txt")

df4 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220528.txt")

df5 = pd.read_csv("http://web.mta.info/developers/data/nyct/turnstile/turnstile_220604.txt")

In [4]:
#After importing weekly data, we have to convert into one table so we use pd.concat function

dfmay = pd.concat([df1,df2, df3, df4, df5], ignore_index=True) 

In [5]:
#Checking for null values as well as the Data types

dfmay.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1055980 entries, 0 to 1055979
Data columns (total 11 columns):
 #   Column                                                                Non-Null Count    Dtype 
---  ------                                                                --------------    ----- 
 0   C/A                                                                   1055980 non-null  object
 1   UNIT                                                                  1055980 non-null  object
 2   SCP                                                                   1055980 non-null  object
 3   STATION                                                               1055980 non-null  object
 4   LINENAME                                                              1055980 non-null  object
 5   DIVISION                                                              1055980 non-null  object
 6   DATE                                                                  1055980 non-

In [6]:
#Since we have 2 different columns as Date and Time, we are merging them into one datetime column 
#Using *datetime* library

datettimes = pd.to_datetime(dfmay["DATE"] + " " + dfmay["TIME"])

In [7]:
#inserting the datetime column into the dataframe

dfmay.insert(loc = 11,column = "datetime", value = datettimes )

In [8]:
#we have 3 different columns regarding the turnstiles so we merge them as well

turnstiles = dfmay["C/A"] + dfmay["UNIT"] + dfmay["SCP"]

In [9]:
#inserting the turnstiles column into our dataframe

dfmay.insert(loc = 12,column = "turnstile", value = turnstiles )

In [10]:
#Dropping irrelevant data from the datafram in order to enhance the efficiency

dfmay.drop(columns = ["C/A", "UNIT", "SCP", "DATE", "DIVISION", "DESC", "LINENAME"], inplace = True)

In [11]:
#checking the dataframe

dfmay.columns = dfmay.columns.str.strip()

In [12]:
#again check for the datatypes since we have made some modifications regarding the dataframe

dfmay.head()

,STATION,TIME,ENTRIES,EXITS,datetime,turnstile
0,59 ST,00:00:00,7706010,2698804,2022-04-30 00:00:00,A002R05102-00-00
1,59 ST,04:00:00,7706014,2698808,2022-04-30 04:00:00,A002R05102-00-00
2,59 ST,08:00:00,7706017,2698835,2022-04-30 08:00:00,A002R05102-00-00
3,59 ST,12:00:00,7706038,2698924,2022-04-30 12:00:00,A002R05102-00-00
4,59 ST,16:00:00,7706084,2698963,2022-04-30 16:00:00,A002R05102-00-00


In [13]:
#since the turnstile data increases cumulatively we have to take the difference between the rows by grouping them by stations

dfmay['net_entry']= dfmay.sort_values(['turnstile','datetime'],ascending = (False, True)).groupby(['turnstile'])['ENTRIES'].diff()
dfmay['net_exits']= dfmay.sort_values(['turnstile','datetime'],ascending = (False, True)).groupby(['turnstile'])['EXITS'].diff()

In [14]:
#these columns are not essential since already calculated the net exit and entry in [12]

dfmay.drop(columns = ["ENTRIES", "EXITS"], inplace = True)

In [15]:
#Another check for the net entries and exits for negative values

dfmay.describe()

,net_entry,net_exits
count,1.050933e+06,1.050933e+06
mean,-7.436687e+02,-2.042623e+03
std,4.221974e+05,2.043506e+06
min,-3.070229e+08,-2.061816e+09
25%,4.000000e+00,6.000000e+00
50%,2.800000e+01,3.800000e+01
75%,9.500000e+01,1.170000e+02
max,5.539190e+05,2.516514e+08


In [16]:
#Eliminating the negative values

dfmay = dfmay[dfmay['net_entry'] >= 0]
dfmay = dfmay[dfmay["net_exits"] >= 0]

In [17]:
#Checking the values again

dfmay.describe()

,net_entry,net_exits
count,1.038005e+06,1.038005e+06
mean,7.926246e+01,9.604153e+01
std,1.499026e+03,1.565741e+03
min,0.000000e+00,0.000000e+00
25%,4.000000e+00,7.000000e+00
50%,2.900000e+01,3.800000e+01
75%,9.600000e+01,1.170000e+02
max,2.621440e+05,9.150600e+05


In [18]:
#let's find the total number of people enters and exits the station

dfmay.insert(loc = 5, column = "total", value = dfmay["net_entry"] + dfmay["net_exits"])

In [19]:
#Whenever a column added we are checking for negative values in essence

dfmay.describe()

,net_entry,total,net_exits
count,1.038005e+06,1.038005e+06,1.038005e+06
mean,7.926246e+01,1.753040e+02,9.604153e+01
std,1.499026e+03,2.199799e+03,1.565741e+03
min,0.000000e+00,0.000000e+00,0.000000e+00
25%,4.000000e+00,1.700000e+01,7.000000e+00
50%,2.900000e+01,9.000000e+01,3.800000e+01
75%,9.600000e+01,2.350000e+02,1.170000e+02
max,2.621440e+05,9.229610e+05,9.150600e+05


In [20]:
#Here is the final version of our dataframe

dfmay

,STATION,TIME,datetime,turnstile,net_entry,total,net_exits
1,59 ST,04:00:00,2022-04-30 04:00:00,A002R05102-00-00,4.0,8.0,4.0
2,59 ST,08:00:00,2022-04-30 08:00:00,A002R05102-00-00,3.0,30.0,27.0
3,59 ST,12:00:00,2022-04-30 12:00:00,A002R05102-00-00,21.0,110.0,89.0
4,59 ST,16:00:00,2022-04-30 16:00:00,A002R05102-00-00,46.0,85.0,39.0
5,59 ST,20:00:00,2022-04-30 20:00:00,A002R05102-00-00,125.0,168.0,43.0
...,...,...,...,...,...,...,...
1055975,RIT-ROOSEVELT,05:00:00,2022-06-03 05:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055976,RIT-ROOSEVELT,09:00:00,2022-06-03 09:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055977,RIT-ROOSEVELT,13:00:00,2022-06-03 13:00:00,TRAM2R46900-05-01,0.0,0.0,0.0
1055978,RIT-ROOSEVELT,17:00:00,2022-06-03 17:00:00,TRAM2R46900-05-01,0.0,2.0,2.0


In [21]:
#Creating a day column in order to check the densities for days of weeks

dfmay.insert(loc = 6, column = "day", value = dfmay['datetime'].dt.day_name())

In [22]:
dfmay

,STATION,TIME,datetime,turnstile,net_entry,total,day,net_exits
1,59 ST,04:00:00,2022-04-30 04:00:00,A002R05102-00-00,4.0,8.0,Saturday,4.0
2,59 ST,08:00:00,2022-04-30 08:00:00,A002R05102-00-00,3.0,30.0,Saturday,27.0
3,59 ST,12:00:00,2022-04-30 12:00:00,A002R05102-00-00,21.0,110.0,Saturday,89.0
4,59 ST,16:00:00,2022-04-30 16:00:00,A002R05102-00-00,46.0,85.0,Saturday,39.0
5,59 ST,20:00:00,2022-04-30 20:00:00,A002R05102-00-00,125.0,168.0,Saturday,43.0
...,...,...,...,...,...,...,...,...
1055975,RIT-ROOSEVELT,05:00:00,2022-06-03 05:00:00,TRAM2R46900-05-01,0.0,0.0,Friday,0.0
1055976,RIT-ROOSEVELT,09:00:00,2022-06-03 09:00:00,TRAM2R46900-05-01,0.0,0.0,Friday,0.0
1055977,RIT-ROOSEVELT,13:00:00,2022-06-03 13:00:00,TRAM2R46900-05-01,0.0,0.0,Friday,0.0
1055978,RIT-ROOSEVELT,17:00:00,2022-06-03 17:00:00,TRAM2R46900-05-01,0.0,2.0,Friday,2.0


In [23]:
dens = dfmay.total.groupby([dfmay["day"],dfmay["STATION"]]).sum()

In [24]:
dens.sort_values()

day        STATION      
Tuesday    ORCHARD BEACH          0.0
Sunday     ORCHARD BEACH          6.0
Saturday   ORCHARD BEACH          7.0
Friday     ORCHARD BEACH         10.0
Monday     ORCHARD BEACH         15.0
                              ...    
Saturday   86 ST            1576122.0
Sunday     86 ST            1795215.0
Monday     86 ST            1975600.0
Tuesday    86 ST            2593150.0
Wednesday  86 ST            2602339.0
Name: total, Length: 2653, dtype: float64

In [25]:
#exporting it in excel form to use in Tableau

#dens.to_excel("günleregöreistasyonlar.xlsx")

In [26]:
dfmay.total.mean()

175.30398312146858

In [45]:
#defining a function to reduce the amount of data

def density_level(total):
    if total > 175:
        return 'high'
    elif  total >= 100:
        return 'medium'
    elif total < 100:
        return 'low'


In [49]:
dfmay.STATION.groupby(dfmay["density_level"]).count()

density_level
high      344137
low       543120
medium    150748
Name: STATION, dtype: int64

In [48]:
dfmay.insert(loc = 8, column = "density_level", value = dfmay.total.apply(density_level))

ValueError: cannot insert density_level, already exists

In [28]:
#dropping the low density rows will make our job easier

dfmay = dfmay[dfmay["density"]=="high"]

In [29]:
#exporting it to excel again to use it with Tableau

dfmay.to_excel("genel.xlsx")

In [30]:
df_best_day = dfmay.groupby(dfmay["day"]).sum()

In [31]:
df_best_day

,net_entry,total,net_exits
day,,,
Friday,9721594.0,22831204.0,13109610.0
Monday,9537674.0,20862663.0,11324989.0
Saturday,5705165.0,12828323.0,7123158.0
Sunday,5039244.0,10959883.0,5920639.0
Thursday,10504837.0,24127734.0,13622897.0
Tuesday,12464434.0,26689031.0,14224597.0
Wednesday,12845591.0,27559246.0,14713655.0


In [32]:
dfmay

,STATION,TIME,datetime,turnstile,net_entry,total,day,density,net_exits
15,59 ST,12:00:00,2022-05-02 12:00:00,A002R05102-00-00,38.0,215.0,Monday,high,177.0
16,59 ST,16:00:00,2022-05-02 16:00:00,A002R05102-00-00,122.0,183.0,Monday,high,61.0
17,59 ST,20:00:00,2022-05-02 20:00:00,A002R05102-00-00,177.0,230.0,Monday,high,53.0
21,59 ST,12:00:00,2022-05-03 12:00:00,A002R05102-00-00,30.0,243.0,Tuesday,high,213.0
22,59 ST,16:00:00,2022-05-03 16:00:00,A002R05102-00-00,137.0,187.0,Tuesday,high,50.0
...,...,...,...,...,...,...,...,...,...
1055804,RIT-ROOSEVELT,17:00:00,2022-06-02 17:00:00,TRAM2R46900-00-01,332.0,335.0,Thursday,high,3.0
1055808,RIT-ROOSEVELT,09:00:00,2022-06-03 09:00:00,TRAM2R46900-00-01,252.0,252.0,Friday,high,0.0
1055809,RIT-ROOSEVELT,13:00:00,2022-06-03 13:00:00,TRAM2R46900-00-01,345.0,349.0,Friday,high,4.0
1055810,RIT-ROOSEVELT,17:00:00,2022-06-03 17:00:00,TRAM2R46900-00-01,325.0,333.0,Friday,high,8.0


In [55]:
df_best_hour = dfmay.groupby(dfmay["TIME"]).sum()

In [56]:
df_best_hour

,net_entry,total,net_exits
TIME,,,
00:00:00,3306556.0,7439980.0,4133424.0
00:00:14,56.0,648.0,592.0
00:00:21,745.0,951.0,206.0
00:00:22,211.0,255.0,44.0
00:00:33,268.0,1141.0,873.0
...,...,...,...
23:58:41,39.0,342.0,303.0
23:59:20,692.0,832.0,140.0
23:59:30,482.0,618.0,136.0
